In [108]:
#导入所需库文件
from numpy import *
from functools import reduce
import re
import csv


In [109]:
# 创建数据集,加载数据
adClass = 1  # 广告,垃圾标识

def loadDataSet():
    '''加载数据集合及其对应的分类'''
    wordList = [['周六', '公司', '一起', '聚餐', '时间'],
                ['优惠', '返利', '打折', '优惠', '金融', '理财'],
    ['喜欢', '机器学习', '一起', '研究', '欢迎', '贝叶斯', '算法', '公式'],
    ['公司', '发票', '税点', '优惠', '增值税', '打折'],
    ['北京', '今天', '雾霾', '不宜', '外出', '时间', '在家', '讨论', '学习'],
    ['招聘', '兼职', '日薪', '保险', '返利']]

    # 1：广告  0：正常
    classVec = [0, 1, 0, 1, 0, 1]
    return wordList, classVec

In [110]:
def doc2VecList(docList):
    """函数说明:数据进行并集操作，最后返回一个词不重复的并集"""
    #reduce(function, iterable[, initializer]): 从左至右积累地应用到 iterable 的条目，以便将该可迭代对象缩减为单一的值
    a = list(reduce(lambda x, y:set(x) | set(y), docList))
    return a  #['','',...,'']



In [111]:
def words2Vec(vecList, inputWords):     #所有词，输入的词组
    '''把单词转化为词向量'''
    dimensions = len(vecList)
    resultVec = [0] * dimensions
    for i in range(dimensions):
        if vecList[i] in inputWords:
            resultVec[i] += 1
    #转化为一维数组
    return array(resultVec)


In [112]:
def trainNB(trainMatrix, trainClass):
    """函数说明:计算，生成每个词对于类别上的概率"""
    # 类别行数——6
    numTrainClass = len(trainClass)
    # 列数——32
    numWords = len(trainMatrix[0])

    '''Laplacian +1平滑'''
    # 全部都初始化为1(全1数组)， 防止出现概率为0的情况
    p0Num = ones(numWords)
    p1Num = ones(numWords)
    # 相应的单词初始化为2
    p0Words = 2.0
    p1Words = 2.0

    # 统计每个分类的词的总数
    for i in range(numTrainClass):
        if trainClass[i] == 1:
            # 数组在对应的位置上相加
            p1Num += trainMatrix[i]
            p1Words += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Words += sum(trainMatrix[i])

    # 计算每种类型里面， 每个单词出现的概率
    # 在计算过程中，由于概率的值较小，于是取对数扩大数值域
    p0Vec = log(p0Num / p0Words)
    p1Vec = log(p1Num / p1Words)

    # 计算在类别中1出现的概率，0出现的概率可通过1-p得到
    pClass1 = sum(trainClass) / float(numTrainClass)
    return p0Vec, p1Vec, pClass1



In [113]:
def classifyNB(testVec, p0Vec, p1Vec, pClass1):
    """分类, 返回分类结果 0 or 1"""
    # 因为概率的值太小了，所以乘法改加法
    # 根据对数特性ln{ P(c)×P(X1|c)×P(X2|c)×...×P(Xn|c) } = lnP(c) + lnP(X1|c) + ... + lnP(Xn|c)
    # 可以简化计算且不失精度
    '''test * pVec已经在trainNB中取过对数了直接相加'''
    p1 = sum(testVec * p1Vec) + log(pClass1)
    p0 = sum(testVec * p0Vec) + log(1 - pClass1)
    if p0 > p1:
        return 0
    return 1



In [114]:
def printClass(words, testClass):
    if testClass == adClass:
        print(words, '推测为：广告邮件')
    else:
        print(words, '推测为：正常邮件')


In [115]:
def tNB():
    # 加载训练数据集
    docList, classVec = loadDataSet()    #单词矩阵、 01向量
    
    # 生成包含所有单词的list
    allWordsVec = doc2VecList(docList)
    
    # 构建词向量矩阵
    '''lambda中的x对应docList的每一行词组'''
    trainMat = list(map(lambda x : words2Vec(allWordsVec, x), docList))   #和docList对应的行(词)向量组
    
    # 训练计算每个词在分类上的概率
    # 其中p0V:每个单词在“非”分类出现的概率， p1V:每个单词在“是”分类出现的概率  pClass1：类别中是1的概率
    p0V, p1V, pClass1 = trainNB(trainMat, classVec)
    
    # 测试数据集
    testwords = ['公司', '聚餐', '讨论', '贝叶斯']
    
    # 转换成单词向量，32个单词构成的数组，如果此单词在数组中，数组的项值置1
    testVec = words2Vec(allWordsVec, testwords)
    
    # 通过将单词向量testVec代入，根据贝叶斯公式，比较各个类别的后验概率，判断当前数据的分类情况
    testClass = classifyNB(testVec, p0V, p1V, pClass1)
    
    # 打印出测试结果
    printClass(testwords, testClass)
    

In [116]:
if __name__ == '__main__':
    tNB()


['公司', '聚餐', '讨论', '贝叶斯'] 推测为：正常邮件
